In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib as plt
import math

# Data Wrangling1

In [2]:
#setting option to display full content rather than truncating at 50 characters
pd.set_option('display.max_colwidth', -1)

### Import Dataset

In [3]:
#import all datasets
df_master = pd.read_csv('data/df_master.csv')
df_name0 = pd.read_csv('data/df_name0.csv')
df_pr0 = pd.read_csv('data/df_pr0.csv')
df_rate0 = pd.read_csv('data/df_rate0.csv')

In [4]:
print(df_master.tail())
print(df_master.columns)

        Unnamed: 0     tconst titleType  \
523859  523859      tt9916622  movie      
523860  523860      tt9916680  movie      
523861  523861      tt9916706  movie      
523862  523862      tt9916730  movie      
523863  523863      tt9916754  movie      

                                                    primaryTitle  isAdult  \
523859  Rodolpho Teóphilo - O Legado de um Pioneiro               0         
523860  De la ilusión al desconcierto: cine colombiano 1970-1995  0         
523861  Dankyavar Danka                                           0         
523862  6 Gunn                                                    0         
523863  Chico Albuquerque - Revelações                            0         

        startYear  runtimeMinutes       genres            directors  \
523859  2015.0    NaN              Documentary  nm9272490,nm9272491   
523860  2007.0     100.0           Documentary  nm0652213             
523861  2013.0    NaN              Comedy       nm7764440        

### Dataset Cleanup Continues

In [5]:
#there is an Unnamed: 0 column that used to be index during Data Wrangling part. We'll remove this
df_master.drop("Unnamed: 0", axis=1, inplace=True)
print(df_master.columns)

Index(['tconst', 'titleType', 'primaryTitle', 'isAdult', 'startYear',
       'runtimeMinutes', 'genres', 'directors', 'writers'],
      dtype='object')


In [6]:
#we know the titleType column of the dataframe is now 'movie' for all rows. We'll remove this
print(df_master.titleType.unique())
df_master.drop("titleType", axis=1, inplace=True)

['movie']


In [7]:
#print(df_master.loc[df_master.startYear.isna()==True])
print(df_master.loc[500:700])

        tconst  \
500  tt0003832   
501  tt0003836   
502  tt0003838   
503  tt0003840   
504  tt0003854   
505  tt0003856   
506  tt0003859   
507  tt0003866   
508  tt0003867   
509  tt0003869   
510  tt0003870   
511  tt0003871   
512  tt0003874   
513  tt0003875   
514  tt0003877   
515  tt0003878   
516  tt0003883   
517  tt0003884   
518  tt0003886   
519  tt0003887   
520  tt0003888   
521  tt0003889   
522  tt0003890   
523  tt0003891   
524  tt0003893   
525  tt0003897   
526  tt0003924   
527  tt0003925   
528  tt0003927   
529  tt0003929   
..         ...   
671  tt0004313   
672  tt0004314   
673  tt0004319   
674  tt0004321   
675  tt0004325   
676  tt0004328   
677  tt0004335   
678  tt0004336   
679  tt0004337   
680  tt0004338   
681  tt0004341   
682  tt0004345   
683  tt0004347   
684  tt0004349   
685  tt0004350   
686  tt0004351   
687  tt0004356   
688  tt0004357   
689  tt0004362   
690  tt0004363   
691  tt0004365   
692  tt0004366   
693  tt0004368   
694  tt000

We can see here that there are around 60000 rows with missing startYear value. But because these data are chronologically ordered we can make an educated assumption that the missing value will be close to the startYear value prior the missing data.

Initially, I thought about taking an avarage of the value prior and after the missing value but in case there are consecutive missing values, this will be difficult.
Is there an easy way?

In [8]:
#forward fill null values by ffill
df_master.startYear.fillna(method='ffill', inplace=True)

In [9]:
#check for null start year rows
df_master.loc[df_master.startYear.isna()==True]
print(df_master.startYear.isna().any())

False


Now we are checking for null in genre

In [10]:
print(df_master.genres.isna().sum())

71824


There are 71824 null values in genres column.  This isn't something we can fill so we will remove them

In [11]:
#removing all null rows with genres column NaN
df_master = df_master.dropna(subset=['genres'])

In [12]:
#after dropping we should be left with 523864-71824 = 452040 rows.
print(df_master.shape)

(452040, 8)


In [13]:
#let's take a look at the current df_master info now.
df_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 452040 entries, 0 to 523863
Data columns (total 8 columns):
tconst            452040 non-null object
primaryTitle      452040 non-null object
isAdult           452040 non-null int64
startYear         452040 non-null float64
runtimeMinutes    305590 non-null float64
genres            452040 non-null object
directors         415123 non-null object
writers           357509 non-null object
dtypes: float64(2), int64(1), object(5)
memory usage: 31.0+ MB


Most of the important columns are without null values such as tconst, title, isAdult, startYear and Genres.
For runtime Minutes, we might be able to do a forward fill since the movie runtime wouldn't have been vastly different between the movies that were out in a similar time.

In [14]:
#finding min and max for runtimeMinutes
print(df_master.loc[df_master.runtimeMinutes.idxmax()])
print(df_master.loc[df_master.runtimeMinutes.idxmin()])


tconst            tt8273150          
primaryTitle      Logistics          
isAdult           0                  
startYear         2012               
runtimeMinutes    51420              
genres            Documentary        
directors         nm9769428,nm8228052
writers           NaN                
Name: 491913, dtype: object
tconst            tt0221032         
primaryTitle      Branding Cattle   
isAdult           0                 
startYear         1898              
runtimeMinutes    1                 
genres            Documentary,Sci-Fi
directors         nm0924920         
writers           NaN               
Name: 131633, dtype: object


This reveals another interesting aspect about the data. Movie with a maximum runtime has 51420 which is 857 hours which is highly unlikely.  Also, the minimum runtime movie is a minute long which is unlikely as well.
There is a need to look into this field a bit more closely.

In [15]:
#extract runtimeminutes column to a series.
runtime_unique = sorted(df_master.runtimeMinutes.dropna().unique().tolist())
print(runtime_unique)

runtime = sorted(df_master.runtimeMinutes.dropna().tolist())
runtime_mean=math.ceil(sum(runtime)/len(runtime))
print(runtime_mean)

[1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0, 16.0, 17.0, 18.0, 19.0, 20.0, 21.0, 22.0, 23.0, 24.0, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0, 35.0, 36.0, 37.0, 38.0, 39.0, 40.0, 41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 53.0, 54.0, 55.0, 56.0, 57.0, 58.0, 59.0, 60.0, 61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 73.0, 74.0, 75.0, 76.0, 77.0, 78.0, 79.0, 80.0, 81.0, 82.0, 83.0, 84.0, 85.0, 86.0, 87.0, 88.0, 89.0, 90.0, 91.0, 92.0, 93.0, 94.0, 95.0, 96.0, 97.0, 98.0, 99.0, 100.0, 101.0, 102.0, 103.0, 104.0, 105.0, 106.0, 107.0, 108.0, 109.0, 110.0, 111.0, 112.0, 113.0, 114.0, 115.0, 116.0, 117.0, 118.0, 119.0, 120.0, 121.0, 122.0, 123.0, 124.0, 125.0, 126.0, 127.0, 128.0, 129.0, 130.0, 131.0, 132.0, 133.0, 134.0, 135.0, 136.0, 137.0, 138.0, 139.0, 140.0, 141.0, 142.0, 143.0, 144.0, 145.0, 146.0, 147.0, 148.0, 149.0, 150.0, 151.0, 152.0, 153.0, 154.0, 155.0, 156.0, 157.0, 158.0, 1

Originally, we were going to forward fill but it looks like the range is too big, ranging from 1min to 51420 minutes which would be outliers.  We will go with mean

In [16]:
#fill null values with mean
df_master.runtimeMinutes.fillna(runtime_mean, inplace=True)

In [17]:
#let's take a look at the info again.  
df_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 452040 entries, 0 to 523863
Data columns (total 8 columns):
tconst            452040 non-null object
primaryTitle      452040 non-null object
isAdult           452040 non-null int64
startYear         452040 non-null float64
runtimeMinutes    452040 non-null float64
genres            452040 non-null object
directors         415123 non-null object
writers           357509 non-null object
dtypes: float64(2), int64(1), object(5)
memory usage: 51.0+ MB


In [18]:
df_master = df_master.dropna(subset=['directors'])

In [19]:
#let's take a look at the info again.  
df_master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 415123 entries, 0 to 523863
Data columns (total 8 columns):
tconst            415123 non-null object
primaryTitle      415123 non-null object
isAdult           415123 non-null int64
startYear         415123 non-null float64
runtimeMinutes    415123 non-null float64
genres            415123 non-null object
directors         415123 non-null object
writers           336491 non-null object
dtypes: float64(2), int64(1), object(5)
memory usage: 28.5+ MB


### Feature Engineering Genres Column

Before we move on, we need to deal with the genres column one more time.  The value for the column is a list of strings containing one or more genre components.
We will feature engineer the dataframe to add a column per genre and denote the genre inclusion by using 1 or 0 if not.

In [20]:
#derive just genres columns to make things easier
genres = df_master.genres
print(genres)
print(len(genres))

#since the index is out of place, we will reset them

genres.reset_index(inplace=True, drop=True)
print(genres)

0         Romance                      
1         Documentary,News,Sport       
2         Biography,Drama              
4         Biography,Crime,Drama        
5         Drama                        
6         Drama                        
7         Drama                        
8         Drama                        
9         Adventure,Fantasy            
10        Drama                        
11        Drama                        
24        Drama                        
26        Drama                        
33        Comedy                       
37        Drama,War                    
39        Documentary                  
47        Drama                        
49        Crime                        
57        Drama,Romance                
59        Adventure,Drama              
63        Drama                        
65        Drama                        
66        Drama                        
67        Biography,Drama,Family       
68        Drama                        


In [21]:
#this list will contain the type of genres available
gen_type=[]

In [22]:
#use to test the code below
test_gen=genres[:100]


#iterate through all elements in the genres series
for genre in genres:
    
    #some of them may contain more than one genre seperated by a comma. 
    if ',' in genre:
        
        #split multiple genres
        items = genre.split(',')
        for item in items:
            
            # having '-' as column names cause issues during our genre assignment in the next block.
            #So we will remove any non alphabet characters
            item=item.replace('-','')
            if item not in gen_type:
                
                gen_type.append(item)
            
    else:
        genre=genre.replace('-','')
        if genre not in gen_type:
            
            gen_type.append(genre)

print(gen_type)

['Romance', 'Documentary', 'News', 'Sport', 'Biography', 'Drama', 'Crime', 'Adventure', 'Fantasy', 'Comedy', 'War', 'Family', 'History', 'SciFi', 'Western', 'Thriller', 'Mystery', 'Horror', 'Action', 'Music', 'Short', 'Animation', 'Musical', 'FilmNoir', 'TalkShow', 'Adult', 'RealityTV', 'GameShow']


gen_type list now contains all of the different genre types available for our dataframe.
All of these different genre types will be a new column added to our dataframe.



In [23]:
test_df=df_master.copy()[:10]

#resets the index
test_df=test_df.reset_index(drop=True)

#this part is adding the columns and populating with 0 for now
for gen in gen_type:
    test_df[gen]=0
    
print(test_df['genres'])
print(test_df.columns)

0    Romance               
1    Documentary,News,Sport
2    Biography,Drama       
3    Biography,Crime,Drama 
4    Drama                 
5    Drama                 
6    Drama                 
7    Drama                 
8    Adventure,Fantasy     
9    Drama                 
Name: genres, dtype: object
Index(['tconst', 'primaryTitle', 'isAdult', 'startYear', 'runtimeMinutes',
       'genres', 'directors', 'writers', 'Romance', 'Documentary', 'News',
       'Sport', 'Biography', 'Drama', 'Crime', 'Adventure', 'Fantasy',
       'Comedy', 'War', 'Family', 'History', 'SciFi', 'Western', 'Thriller',
       'Mystery', 'Horror', 'Action', 'Music', 'Short', 'Animation', 'Musical',
       'FilmNoir', 'TalkShow', 'Adult', 'RealityTV', 'GameShow'],
      dtype='object')


Here, we will write a function that will iterate through all rows in dataframe to read the genres associated with each row then assign a value to corresponding column we added above.

In [24]:
def set_genre(df):

    '''This method takes in a dataframe and parses the genres column, 
    determines genre associated with each rows and assigns 1 to the correct genre column'''

    #using iterrows, we are iterating through all rows in the dataframe
    for index,row in df.iterrows():
        
        genre = row.genres.split(',')
        #print(genre)
        #print(df.iloc[index]['genres'])
       # df.Musical[index]=1
        
        for gen in genre:
            
            #we'll need to remove the '-' first
            gen=gen.replace('-','')
            
            if gen == 'Romance':
 
                df.Romance[index]=1
                
            elif gen == 'Documentary':

                df.Documentary[index] =1
            elif gen == 'News':
                df.News[index]=1
            elif gen == 'Sport':
                df.Sport[index]=1
            elif gen == 'Biography':
                df.Biography[index]=1
            elif gen == 'Drama':
                df.Drama[index]=1
            elif gen == 'Crime':
                df.Crime[index]=1
            elif gen == 'Adventure':
                df.Adventure[index]=1
            elif gen == 'Fantasy':
                df.Fantasy[index]=1
            elif gen == 'Comedy':
                df.Comedy[index]=1
            elif gen == 'War':
                df.War[index]=1
            elif gen =='Family':
                df.Family[index]=1
            elif gen == 'History':
                df.History[index]=1
            elif gen == 'SciFi':
                df.SciFi[index]=1
            elif gen == 'Western':
                df.Western[index]=1
            elif gen == 'Thriller':
                df.Thriller[index]=1
            elif gen == 'Mystery':
                df.Mystery[index]=1
            elif gen == 'Horror':
                df.Horror[index]=1
            elif gen == 'Action':
                df.Action[index]=1
            elif gen == 'Music':
                df.Music[index]=1
            elif gen == 'Short':
                df.Short[index]=1
            elif gen == 'Animation':
                df.Animation[index]=1
            elif gen =='Musical':
                df.Musical[index]=1
            elif gen == 'FilmNoir':
                df.FilmNoir[index]=1
            elif gen == 'TalkShow':
                df.TalkShow[index]=1
            elif gen == 'Adult':
                df.Adult[index]=1
            elif gen =='RealityTV':
                df.RealityTV[index]=1
            elif gen =='GameShow':
                df.GameShow[index]=1
            else:
                print('something is not right: '+gen)
    return df

    
#we'll test with the test_df first
set_genre(test_df)
print(test_df['genres'])
print(test_df.loc[:,['Romance','Documentary','News','Sport','Drama','Biography','Crime','Fantasy','Adventure']])

/Users/byungchankim/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/byungchankim/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/byungchankim/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/byungchankim/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:29: SettingWithCopyWarning

0    Romance               
1    Documentary,News,Sport
2    Biography,Drama       
3    Biography,Crime,Drama 
4    Drama                 
5    Drama                 
6    Drama                 
7    Drama                 
8    Adventure,Fantasy     
9    Drama                 
Name: genres, dtype: object
   Romance  Documentary  News  Sport  Drama  Biography  Crime  Fantasy  \
0  1        0            0     0      0      0          0      0         
1  0        1            1     1      0      0          0      0         
2  0        0            0     0      1      1          0      0         
3  0        0            0     0      1      1          1      0         
4  0        0            0     0      1      0          0      0         
5  0        0            0     0      1      0          0      0         
6  0        0            0     0      1      0          0      0         
7  0        0            0     0      1      0          0      0         
8  0        0            0

/Users/byungchankim/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


It looks like the method is working correctly.  Printing the genres and the feature engineered columns together shows that the translation was done successfully.

Now we will do this to the df_master. Currently df_master has not undergone any column addition. Let's start with that.

In [25]:
print(df_master.info())
print(df_master.head())

print(gen_type)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 415123 entries, 0 to 523863
Data columns (total 8 columns):
tconst            415123 non-null object
primaryTitle      415123 non-null object
isAdult           415123 non-null int64
startYear         415123 non-null float64
runtimeMinutes    415123 non-null float64
genres            415123 non-null object
directors         415123 non-null object
writers           336491 non-null object
dtypes: float64(2), int64(1), object(5)
memory usage: 28.5+ MB
None
      tconst                   primaryTitle  isAdult  startYear  \
0  tt0000009  Miss Jerry                     0        1894.0      
1  tt0000147  The Corbett-Fitzsimmons Fight  0        1897.0      
2  tt0000335  Soldiers of the Cross          0        1900.0      
4  tt0000574  The Story of the Kelly Gang    0        1906.0      
5  tt0000615  Robbery Under Arms             0        1907.0      

   runtimeMinutes                  genres            directors  \
0  45.0            Roman

In [26]:
#to make things tidier, we will start by resetting the index
df_master=df_master.reset_index(drop=True)

#this part is adding the columns and populating with 0 for now
#gen_type list is already created for us. We can reuse this.
for gen in gen_type:
    df_master[gen]=0
    
print(df_master.columns)
print(df_master.info())

Index(['tconst', 'primaryTitle', 'isAdult', 'startYear', 'runtimeMinutes',
       'genres', 'directors', 'writers', 'Romance', 'Documentary', 'News',
       'Sport', 'Biography', 'Drama', 'Crime', 'Adventure', 'Fantasy',
       'Comedy', 'War', 'Family', 'History', 'SciFi', 'Western', 'Thriller',
       'Mystery', 'Horror', 'Action', 'Music', 'Short', 'Animation', 'Musical',
       'FilmNoir', 'TalkShow', 'Adult', 'RealityTV', 'GameShow'],
      dtype='object')
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 415123 entries, 0 to 415122
Data columns (total 36 columns):
tconst            415123 non-null object
primaryTitle      415123 non-null object
isAdult           415123 non-null int64
startYear         415123 non-null float64
runtimeMinutes    415123 non-null float64
genres            415123 non-null object
directors         415123 non-null object
writers           336491 non-null object
Romance           415123 non-null int64
Documentary       415123 non-null int64
News          

All genres have been successfully added to the dataframe.  Next up, translating genres column to our feature engineered columns.

In [28]:
df_copy = df_master.copy()

df_copy = df_copy[:1000]
print(df_copy.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 36 columns):
tconst            1000 non-null object
primaryTitle      1000 non-null object
isAdult           1000 non-null int64
startYear         1000 non-null float64
runtimeMinutes    1000 non-null float64
genres            1000 non-null object
directors         1000 non-null object
writers           871 non-null object
Romance           1000 non-null int64
Documentary       1000 non-null int64
News              1000 non-null int64
Sport             1000 non-null int64
Biography         1000 non-null int64
Drama             1000 non-null int64
Crime             1000 non-null int64
Adventure         1000 non-null int64
Fantasy           1000 non-null int64
Comedy            1000 non-null int64
War               1000 non-null int64
Family            1000 non-null int64
History           1000 non-null int64
SciFi             1000 non-null int64
Western           1000 non-null int64
Thriller    

In [29]:
#set_genre method has already been declared and tested above. We'll just call the method.
set_genre(df_copy)

#print(df_copy.loc[100:110])

/Users/byungchankim/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/byungchankim/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/byungchankim/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/byungchankim/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning

,tconst,primaryTitle,isAdult,startYear,runtimeMinutes,genres,directors,writers,Romance,Documentary,...,Action,Music,Short,Animation,Musical,FilmNoir,TalkShow,Adult,RealityTV,GameShow
0,tt0000009,Miss Jerry,0,1894.0,45.0,Romance,nm0085156,nm0085156,1,0,...,0,0,0,0,0,0,0,0,0,0
1,tt0000147,The Corbett-Fitzsimmons Fight,0,1897.0,20.0,"Documentary,News,Sport",nm0714557,NaN,0,1,...,0,0,0,0,0,0,0,0,0,0
2,tt0000335,Soldiers of the Cross,0,1900.0,89.0,"Biography,Drama","nm0095714,nm0675140",NaN,0,0,...,0,0,0,0,0,0,0,0,0,0
3,tt0000574,The Story of the Kelly Gang,0,1906.0,70.0,"Biography,Crime,Drama",nm0846879,nm0846879,0,0,...,0,0,0,0,0,0,0,0,0,0
4,tt0000615,Robbery Under Arms,0,1907.0,89.0,Drama,nm0533958,"nm0092809,nm0533958",0,0,...,0,0,0,0,0,0,0,0,0,0
5,tt0000630,Hamlet,0,1908.0,89.0,Drama,nm0143333,nm0000636,0,0,...,0,0,0,0,0,0,0,0,0,0
6,tt0000675,Don Quijote,0,1908.0,89.0,Drama,nm0194088,nm0148859,0,0,...,0,0,0,0,0,0,0,0,0,0
7,tt0000676,Don Álvaro o la fuerza del sino,0,1908.0,89.0,Drama,nm0194088,nm0729044,0,0,...,0,0,0,0,0,0,0,0,0,0
8,tt0000679,The Fairylogue and Radio-Plays,0,1908.0,120.0,"Adventure,Fantasy","nm0091767,nm0877783","nm0000875,nm0877783",0,0,...,0,0,0,0,0,0,0,0,0,0
9,tt0000739,El pastorcito de Torrente,0,1908.0,89.0,Drama,nm0191133,NaN,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
#At this point, we have run into an unusual issue using jupyter notebook.
#runing the set_genre method on a dataframe with more than 10000 records at one time will cause the compiler to hang.
#We have tested above that 10000 records works.
#So we will divide the dataframe into chunks of 10000 records and later concatenate them together. 

#df_test used for test
#df_test = df_copy.copy()[:2567]

n = 1000  #chunk row size

#list comprehension to divide the dataframe
list_df = [df_master[i:i+n] for i in range(0,df_master.shape[0],n)]

#test to make sure the all data are captured until the end
print(list_df[415].tconst.tail())
print(df_master.tconst.tail())

print(len(list_df))


415118    tt9916538
415119    tt9916622
415120    tt9916680
415121    tt9916706
415122    tt9916754
Name: tconst, dtype: object
415118    tt9916538
415119    tt9916622
415120    tt9916680
415121    tt9916706
415122    tt9916754
Name: tconst, dtype: object
416


In [37]:
#Now that the list of sub dataframes are ready we can itercall set_genre on each sub dataframes.

for i in range(len(list_df)):
    list_df[i]=set_genre(list_df[i])
    print('out of ' + str(len(list_df)) +', '+str(i)+' dataframe finished')
    

#concatenating the sub dataframes back together.
df_concat= pd.concat(list_df)

/Users/byungchankim/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/byungchankim/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/Users/byungchankim/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/byungchankim/anaconda3/lib/python3.7/site-packages/ipykernel

out of 416, 0 dataframe finished


/Users/byungchankim/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/byungchankim/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


out of 416, 1 dataframe finished
out of 416, 2 dataframe finished


/Users/byungchankim/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:67: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


...

out of 416, 41 dataframe finished
out of 416, 42 dataframe finished
out of 416, 43 dataframe finished
out of 416, 44 dataframe finished
out of 416, 45 dataframe finished
out of 416, 46 dataframe finished


/Users/byungchankim/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:75: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


out of 416, 47 dataframe finished
out of 416, 48 dataframe finished
out of 416, 49 dataframe finished
out of 416, 50 dataframe finished
out of 416, 51 dataframe finished
out of 416, 52 dataframe finished
out of 416, 53 dataframe finished
out of 416, 54 dataframe finished
out of 416, 55 dataframe finished
out of 416, 56 dataframe finished
out of 416, 57 dataframe finished
out of 416, 58 dataframe finished
out of 416, 59 dataframe finished
out of 416, 60 dataframe finished
out of 416, 61 dataframe finished
out of 416, 62 dataframe finished
out of 416, 63 dataframe finished
out of 416, 64 dataframe finished
out of 416, 65 dataframe finished
out of 416, 66 dataframe finished
out of 416, 67 dataframe finished
out of 416, 68 dataframe finished
out of 416, 69 dataframe finished
out of 416, 70 dataframe finished
out of 416, 71 dataframe finished
out of 416, 72 dataframe finished
out of 416, 73 dataframe finished
out of 416, 74 dataframe finished
out of 416, 75 dataframe finished
out of 416, 76

/Users/byungchankim/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:77: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


out of 416, 243 dataframe finished
out of 416, 244 dataframe finished
out of 416, 245 dataframe finished
out of 416, 246 dataframe finished
out of 416, 247 dataframe finished
out of 416, 248 dataframe finished
out of 416, 249 dataframe finished
out of 416, 250 dataframe finished
out of 416, 251 dataframe finished
out of 416, 252 dataframe finished
out of 416, 253 dataframe finished
out of 416, 254 dataframe finished
out of 416, 255 dataframe finished
out of 416, 256 dataframe finished
out of 416, 257 dataframe finished
out of 416, 258 dataframe finished
out of 416, 259 dataframe finished
out of 416, 260 dataframe finished
out of 416, 261 dataframe finished
out of 416, 262 dataframe finished
out of 416, 263 dataframe finished
out of 416, 264 dataframe finished
out of 416, 265 dataframe finished
out of 416, 266 dataframe finished
out of 416, 267 dataframe finished
out of 416, 268 dataframe finished
out of 416, 269 dataframe finished
out of 416, 270 dataframe finished
out of 416, 271 data

It took around 21 hours to complete this translation.

In [38]:
#Let's save the dataframes to csv to be used in Data Storytelling

df_concat.to_csv(r'/Users/byungchankim/Downloads/Springboard/capstone1/data/df_concat.csv')

In [40]:
df_concat.tail()

,tconst,primaryTitle,isAdult,startYear,runtimeMinutes,genres,directors,writers,Romance,Documentary,...,Action,Music,Short,Animation,Musical,FilmNoir,TalkShow,Adult,RealityTV,GameShow
415118,tt9916538,Kuambil Lagi Hatiku,0,2019.0,123.0,Drama,nm8185151,"nm4843252,nm4900525,nm2679404",0,0,...,0,0,0,0,0,0,0,0,0,0
415119,tt9916622,Rodolpho Teóphilo - O Legado de um Pioneiro,0,2015.0,89.0,Documentary,"nm9272490,nm9272491","nm9272490,nm9272491",0,1,...,0,0,0,0,0,0,0,0,0,0
415120,tt9916680,De la ilusión al desconcierto: cine colombiano 1970-1995,0,2007.0,100.0,Documentary,nm0652213,"nm0652213,nm10538576",0,1,...,0,0,0,0,0,0,0,0,0,0
415121,tt9916706,Dankyavar Danka,0,2013.0,89.0,Comedy,nm7764440,nm7933903,0,0,...,0,0,0,0,0,0,0,0,0,0
415122,tt9916754,Chico Albuquerque - Revelações,0,2013.0,89.0,Documentary,"nm9272490,nm8349149","nm8349149,nm9272490",0,1,...,0,0,0,0,0,0,0,0,0,0


Now we are left with 415123 rows.  But there is one more row drops that we need to do. That is dropping all rows of movie data without any ratings.  Ratings is a key feature in our project and we'll need to have this down the road.
However, this does not mean that the movies without rating is of no use.  We'll still keep a copy of the full dataframe during data visualization part of the project.

In [42]:
#make a copy of df_master
df_concat_rate = pd.merge(df_concat,df_rate0,on='tconst')

In [44]:
print(len(df_rate0['tconst'].unique()))
df_concat_rate.info()

236124
<class 'pandas.core.frame.DataFrame'>
Int64Index: 222860 entries, 0 to 222859
Data columns (total 39 columns):
tconst            222860 non-null object
primaryTitle      222860 non-null object
isAdult           222860 non-null int64
startYear         222860 non-null float64
runtimeMinutes    222860 non-null float64
genres            222860 non-null object
directors         222860 non-null object
writers           198671 non-null object
Romance           222860 non-null int64
Documentary       222860 non-null int64
News              222860 non-null int64
Sport             222860 non-null int64
Biography         222860 non-null int64
Drama             222860 non-null int64
Crime             222860 non-null int64
Adventure         222860 non-null int64
Fantasy           222860 non-null int64
Comedy            222860 non-null int64
War               222860 non-null int64
Family            222860 non-null int64
History           222860 non-null int64
SciFi             222860 non-null

The reason we are seeing 222860 rows rather than 236124 after the merge is because this is an inner join.  And by removing rows that has null Genres or Directors, we have removed some movie data that had ratings available.  
Thankfully is it not that much.

In [45]:
# Unnamed: 0 column from df_rate0 was carried over. We will drop this.
df_concat_rate.drop("Unnamed: 0", axis=1, inplace=True)

In [ ]:
#Let's save the dataframe to be used in Data Storytelling
df_concat_rate.to_csv(r'/Users/byungchankim/Downloads/Springboard/capstone1/data/df_concat_rate.csv')